In [111]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import re



In [ ]:
import re

def clean_name(name):
    name = name.lower()

    intel_match = re.search(r"(intel core i[3579] \d{4,5}[k|kf|f]?)", name)
    if intel_match:
        return intel_match.group(1)

    amd_match = re.search(r"(amd ryzen [3579] \d{3,4}[x|xt|x3d|g|gt|f]?)", name)
    if amd_match:
        return amd_match.group(1)

    return name


In [113]:
cpu_spec = pd.read_csv('./data/raw/ppp/cpu.csv')
cpu_price = pd.read_csv('./data/filtered/prices_processor.csv')

In [114]:
cpu_price['clean_name'] = cpu_price['item_name'].apply(clean_name)
cpu_price.head()

,item_name,category_name,item_price_in_php,stocks,clean_name
0,"Intel Core i3 10105F 4C/8T 6mb 65W LGA1200, Box",Processor,4999.0,5,intel core i3 10105f
1,Intel Core i5 10400 6C/12T 12mb UHD630 65W LGA...,Processor,7699.0,0,intel core i5 10400
2,"Intel Core i3 13100F 4C/8T 12mb 58W LGA1700, Box",Processor,5799.0,16,intel core i3 13100f
3,Intel Core i3 14100 4C/8T 12mb UHD730 60W LGA1...,Processor,8699.0,8,intel core i3 14100
4,"Intel Core i3 14100F 4C/8T 12mb 58W LGA1700, Box",Processor,6999.0,6,intel core i3 14100f


In [115]:
cpu_spec['clean_name'] = cpu_spec['name'].apply(clean_name)

cpu_spec.head()

,name,price,core_count,core_clock,boost_clock,microarchitecture,tdp,graphics,clean_name
0,AMD Ryzen 7 9800X3D,451.50,8,4.7,5.2,Zen 5,120,Radeon,amd ryzen 7 9800x
1,AMD Ryzen 7 7800X3D,340.05,8,4.2,5.0,Zen 4,120,Radeon,amd ryzen 7 7800x
2,AMD Ryzen 5 7600X,170.49,6,4.7,5.3,Zen 4,105,Radeon,amd ryzen 5 7600x
3,AMD Ryzen 5 9600X,204.99,6,3.9,5.4,Zen 5,65,Radeon,amd ryzen 5 9600x
4,AMD Ryzen 7 7700X,242.98,8,4.5,5.4,Zen 4,105,Radeon,amd ryzen 7 7700x


cpu price + spec match

In [117]:
cpu_spec['clean_name'] = cpu_spec['name'].apply(clean_name)
cpu_price['clean_name'] = cpu_price['item_name'].apply(clean_name)

cpu_spec.to_csv('data/processed/cpuS.csv', index = False)
cpu_price.to_csv('data/processed/cpuP.csv', index = False)


In [ ]:
for spec_name in cpu_spec_names:
    match = process.extractOne(spec_name, cpu_price_names, scorer=fuzz.token_sort_ratio)
    if match and match[1] > 40:
        print(f"{spec_name} <---> {match[0]} (score: {match[1]})")


In [97]:
merged_cpus = cpu_spec.merge(cpu_price[['clean_name', 'item_price_in_php']],
                             on='clean_name', how='inner')
merged_cpus.head()

merged_cpus.to_csv('data/processed/cpu.csv', index = False)
